# Ranking documents using skip grams word embedding model for phrase queries

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
import gensim 
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

## Setup
#### The corpus/documents are extracted from the pickle files.
#### Stemmer has been initialised to convert term to its root form (Example : received -> receive)

In [3]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path(".")

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

## Model

Skip-gram model is a type of neural network architecture that is commonly used for word embedding. It is a form of unsupervised learning that aims to learn the distributional representation of words.The basic idea behind skip-gram model is to predict the context words surrounding a given target word, rather than predicting the target word from the context words. The context words are defined as the words that occur within a certain window of the target word.

The skip-gram model consists of an input layer, a hidden layer, and an output layer. The input layer represents the target word, and the output layer represents the context words. The hidden layer is used to transform the input word into a distributed representation, or embedding, which is used to predict the context words.During training, the skip-gram model is fed with a large corpus of text. For each target word in the corpus, a training example is created by randomly selecting one of its context words as the output, and setting the remaining context words as inputs to the model. The model is then trained to predict the output context word given the input context words.

The training process involves adjusting the weights of the model to minimize the difference between the predicted context word and the actual context word. The weights are adjusted using backpropagation algorithm, which updates the weights based on the difference between the predicted and actual output.After training, the skip-gram model generates a dense vector representation for each word in the vocabulary. The vector representation captures the semantic and syntactic information of the word, and can be used as input to other machine learning models for various natural language processing tasks such as text classification, information retrieval, and machine translation.

## Building the model

The following code initializes an empty list called data, which will be used to store the tokenized and preprocessed documents. Then, it loops over each document in the documents list and tokenizes each sentence into a list of words. Each word is then converted to lowercase and added to a temporary list called temp. The temp list is then appended to the data list, which now contains a list of tokenized and preprocessed documents.

Next, the Word2Vec model from the gensim library is used to train a word embedding model on the data list. The min_count parameter specifies the minimum frequency count of a word in the corpus for it to be included in the model. The vector_size parameter specifies the dimensionality of the word vectors. The window parameter specifies the maximum distance between the target word and the context word within a sentence. The sg parameter specifies the training algorithm to be used - 1 for skip-gram and 0 for CBOW. In this case, sg is set to 1 which corresponds to the skip-gram algorithm for training.

After training, the word embedding model skipgram_model contains dense vector representations for each word in the corpus. These vector representations can be used for various natural language processing tasks such as text classification, information retrieval, and machine translation.

In [4]:
data = list()
for i in documents: 
    temp = list()
    # tokenize the sentence into words 
    for j in word_tokenize(i[0]): 
        temp.append(j.lower()) 
    data.append(temp)

skipgram_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5,sg = 1)

## Generating word embeddings for documents

The following code defines two functions.

1.**preprocess(text)** takes a string of text as input, tokenizes it, removes stop words and filters out words with length less than two. It returns a list of preprocessed tokens.

2.**text_embedding(text, model)** takes a string of text and a pre-trained word embedding model as inputs. It preprocesses the text using the preprocess() function, obtains the word embeddings for each word in the preprocessed text from the pre-trained word embedding model, takes the average of the word embeddings to generate a document embedding, and returns the document embedding as a numpy array.

Finally, the **document_vecs** variable is assigned a list of document embeddings generated using the text_embedding() function and the pre-trained word embedding model (skipgram_model) for each document in the documents list.


In [5]:
def preprocess(text):
    tokens = []
    for word in word_tokenize(text, language='english'):
        if len(word) >= 2 and word not in stop_words:
            tokens.append(word)
    return tokens

# Define a function to generate text embeddings
def text_embedding(text, model):
    # Preprocess the text (tokenize, remove stop words, stem)
    preprocessed_text = preprocess(text)
    # Get the word embeddings for each word in the document
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    # Take the average of the word embeddings to generate a document embedding
    if len(word_embeddings) > 0:
        document_embedding = np.mean(word_embeddings, axis=0)
    else:
        document_embedding = np.zeros(model.vector_size)
    return document_embedding

document_vecs = [text_embedding(document[0], skipgram_model) for document in documents]

In [6]:
pd.DataFrame(document_vecs)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.011627,0.009185,0.010881,0.305174,0.099559,-0.301031,-0.069827,0.323303,0.146935,-0.055789,...,-0.026393,0.053196,0.072542,0.213979,-0.195234,0.186686,-0.005739,0.199159,0.062985,-0.040922
1,-0.002238,-0.128637,0.078069,0.244151,0.094506,-0.409146,-0.024416,0.433748,0.129218,-0.003537,...,0.111575,0.067492,0.209744,0.345303,-0.104026,0.268556,0.013777,0.137338,0.069604,-0.129610
2,0.083339,0.001652,0.008078,0.181379,0.252945,-0.332915,-0.099393,0.383637,0.089563,0.082441,...,0.005498,0.066760,0.139600,0.303438,-0.232925,0.278231,0.172427,0.373989,-0.056544,-0.159889
3,0.047034,0.022647,0.036241,0.206683,0.247140,-0.298477,-0.072399,0.305348,0.101278,-0.010541,...,0.027765,0.142191,0.191782,0.191793,-0.280872,0.233433,0.200667,0.338396,-0.071003,-0.144382
4,0.011981,0.023402,0.040755,0.258638,0.256489,-0.288979,-0.176186,0.269207,0.097166,0.054173,...,0.104881,0.059481,0.202893,0.228863,-0.227901,0.228968,0.180517,0.371005,-0.065681,-0.239247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6065,-0.004867,-0.030920,0.059274,0.093520,0.104142,-0.234052,-0.030891,0.305300,0.167307,-0.160520,...,0.052018,0.071726,0.079872,0.129999,-0.091480,0.375989,0.048597,0.182046,0.038663,-0.091841
6066,-0.081024,0.027247,0.068879,0.218075,0.149788,-0.190551,-0.061587,0.236786,0.056047,-0.115653,...,-0.008159,0.038606,0.086236,0.190957,-0.096757,0.225651,0.030694,0.155867,0.097484,-0.064644
6067,-0.122089,-0.091714,0.082900,0.078946,0.024101,-0.251328,0.048014,0.143938,0.049801,-0.019700,...,0.064424,-0.035818,0.101320,0.209992,0.164360,0.355735,0.112376,-0.090869,0.044574,0.028526
6068,-0.121006,0.091159,0.004376,0.277010,-0.122839,-0.091282,0.028686,0.242289,0.242448,-0.007706,...,-0.034037,-0.019213,0.083702,0.086762,0.186181,0.161484,-0.080373,-0.090528,0.332764,0.077187


## Similarity measure

Here cosine_similarity is used to compare the query vector and and the document vector.

In [7]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

## Saving document vectors for future use

In [8]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'wb')
pickle.dump(document_vecs, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'rb')     
document_vecs = pickle.load(dbfile)
dbfile.close()

## Processing the query

This code calculates the similarity scores between a given query and a set of documents. It first generates a document vector for each document in the set using the text_embedding function. Then, it generates a query vector using the same function. The cosine similarity between the query vector and each document vector is calculated and stored in the similarity_scores list. Finally, the documents are ranked in descending order based on their similarity scores with the query, and the top five documents are printed.

In [9]:
# Example usage: compare a query with a set of documents
query = "Excluded driver"

query_vec = text_embedding(query,skipgram_model)
# Calculate similarity scores between the query and all documents
similarity_scores = [cosine_similarity(query_vec, document_vec) for document_vec in document_vecs]
# Rank documents based on similarity scores
ranked_documents = [document for _, document in sorted(zip(similarity_scores, documents), reverse=True)]
# Print the ranked documents
ranked_documents[0:5]

[('any person , other than you or a resident relative , while using a non-owned auto : which is available for hire by the public , or separate premium is charged for each auto . ',
  '.\\Docs\\Auto\\AU127-1.docx'),
 ('an unidentified automobile is one whose owner or driver can not be determined . ',
  '.\\Docs\\Auto\\1215E.2.docx'),
 ('for purposes of this policy , a private passenger type auto shall be deemed to be owned by a any of the following types of vehicles on the date you become the owner : person if leased : a. a private passenger auto ; or under a written agreement to that person ; b. a pickup or van . ',
  '.\\Docs\\Auto\\PL-600003-87.docx'),
 ('your driver ’ s license or auto registration has been under suspension or revocation during the policy period . ',
  '.\\Docs\\Auto\\7thEditionPolicy.docx'),
 ('m enot owned by you or a relative who resides in your household , being operated by you with the owner ’ s permission . ',
  '.\\Docs\\Auto\\AU127-1.docx')]